In [1]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [2]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [3]:
def getInfo(filesGrp, img_path):

  dslabel = {
    "id":"",
    "11originalFileJPG":"",
    "11originalFileGrp":[],
    "GeoTagInfo":{
      "coords":[],
      "datetime_original":"",
      "gps_altitude":0,
      "gps_altitude_ref":""    
    },
    "label":[]
  }

# Remove Drive letter with root path
  dslabel['11originalFileJPG'] = img_path[3:]
  dslabel['11originalFileGrp'] = [x[3:] for x in filesGrp  if ('.JPG' not in x)]

  with open(img_path, 'rb') as src:
    img = exifimg(src)

  coords = [
    decimal_coords(img.gps_longitude,
    img.gps_longitude_ref),
      decimal_coords(img.gps_latitude,
    img.gps_latitude_ref)
  ]

  dslabel['id'] = img.datetime_original.replace(" ","").replace(":","") + "_{:.10f}".format(coords[0]).replace(".","") + "_{:.10f}".format(coords[1]).replace(".","") + "_{:f}".format(img.gps_altitude).replace(".","")
  dslabel['GeoTagInfo']['coords'] = coords
  dslabel['GeoTagInfo']['datetime_original'] = img.datetime_original
  dslabel['GeoTagInfo']['gps_altitude'] = img.gps_altitude
  dslabel['GeoTagInfo']['gps_altitude_ref'] = img.gps_altitude_ref
  
  return dslabel

In [8]:
src_folder = r'D:\G07_청도그레이스'
out_folder = r'D:\ToAWS2'

course_list = ['청도','포항','중문','시흥','거창','속리산','춘천','오너스']

course_ids = {
  '포항':'MGC001',
  '청도':'MGC002',
  '중문':'MGC003',
  '시흥':'MGC004',
  '거창':'MGC005',
  '속리산':'MGC006',
  '춘천':'MGC007',
  '오너스':'MGC007'
}

folder_type = ['rgb','ndvi','thumb']

In [9]:
course_id = course_ids.get('청도')
course_id

'MGC002'

In [10]:
files = glob.glob(os.path.join(src_folder,'**/*.JPG'), recursive=True)
files = [x for x in files if 'DJI' in x]

len(files)

117

In [11]:
files[0]

'D:\\G07_청도그레이스\\CU_근접촬영\\02_물리적손상잔디\\20_고온피해\\G07_CU_0220_DJI_20230823162127_0005_D.JPG'

In [12]:
file_ = files[0]
path = os.path.normpath(file_)
info = path.split(os.sep)
info

['D:',
 'G07_청도그레이스',
 'CU_근접촬영',
 '02_물리적손상잔디',
 '20_고온피해',
 'G07_CU_0220_DJI_20230823162127_0005_D.JPG']

In [13]:
os.path.join(os.path.split(file_)[0],(('_').join(os.path.split(file_)[1].split('_')[:-2])[:-3]+"***_" + os.path.split(file_)[1].split('_')[-2]) + '*')

'D:\\G07_청도그레이스\\CU_근접촬영\\02_물리적손상잔디\\20_고온피해\\G07_CU_0220_DJI_20230823162***_0005*'

In [14]:
glob.glob(os.path.join(os.path.split(file_)[0],(('_').join(os.path.split(file_)[1].split('_')[:-2])[:-3]+"***_" + os.path.split(file_)[1].split('_')[-2]) + '*'), recursive=True)

['D:\\G07_청도그레이스\\CU_근접촬영\\02_물리적손상잔디\\20_고온피해\\G07_CU_0220_DJI_20230823162127_0005_D.JPG',
 'D:\\G07_청도그레이스\\CU_근접촬영\\02_물리적손상잔디\\20_고온피해\\G07_CU_0220_DJI_20230823162127_0005_MS_G.TIF',
 'D:\\G07_청도그레이스\\CU_근접촬영\\02_물리적손상잔디\\20_고온피해\\G07_CU_0220_DJI_20230823162127_0005_MS_NIR.TIF',
 'D:\\G07_청도그레이스\\CU_근접촬영\\02_물리적손상잔디\\20_고온피해\\G07_CU_0220_DJI_20230823162127_0005_MS_R.TIF',
 'D:\\G07_청도그레이스\\CU_근접촬영\\02_물리적손상잔디\\20_고온피해\\G07_CU_0220_DJI_20230823162127_0005_MS_RE.TIF']

In [15]:
InfoJsonArray = []
target_folder = []

for idx, file_ in enumerate(files):

  course = list(filter(lambda y: y in file_, course_list))[0]

  path = os.path.normpath(file_)
  info = path.split(os.sep)

  filesGrp = glob.glob(os.path.join(os.path.split(file_)[0],(('_').join(os.path.split(file_)[1].split('_')[:-2])[:-3]+"***_" + os.path.split(file_)[1].split('_')[-2]) + '*'), recursive=True)
  print(file_)
  file_info = getInfo(filesGrp, file_)

  file_info['label'] = {
    "level1":info[-3],
    "level2":info[-2],
    "level3":"",
    "TurfType":""
  }

  InfoJsonArray.append( file_info)
  
#   print(save_name)

#   print('{} out of {} files Completed'.format(idx, len(target_files)))

save_name = os.path.join(out_folder, 'afterLabeldata'+course_id+'.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(InfoJsonArray, final , ensure_ascii=False)



  

D:\G07_청도그레이스\CU_근접촬영\02_물리적손상잔디\20_고온피해\G07_CU_0220_DJI_20230823162127_0005_D.JPG
D:\G07_청도그레이스\CU_근접촬영\02_물리적손상잔디\20_고온피해\G07_CU_0220_DJI_20230823162129_0006_D.JPG
D:\G07_청도그레이스\CU_근접촬영\02_물리적손상잔디\20_고온피해\G07_CU_0220_DJI_20230823162131_0007_D.JPG
D:\G07_청도그레이스\CU_근접촬영\02_물리적손상잔디\20_고온피해\G07_CU_0220_DJI_20230823162133_0008_D.JPG
D:\G07_청도그레이스\CU_근접촬영\02_물리적손상잔디\20_고온피해\G07_CU_0220_DJI_20230823162135_0009_D.JPG
D:\G07_청도그레이스\CU_근접촬영\02_물리적손상잔디\20_고온피해\G07_CU_0220_DJI_20230823162137_0010_D.JPG
D:\G07_청도그레이스\CU_근접촬영\02_물리적손상잔디\20_고온피해\G07_CU_0220_DJI_20230823162139_0011_D.JPG
D:\G07_청도그레이스\CU_근접촬영\02_물리적손상잔디\20_고온피해\G07_CU_0220_DJI_20230823162141_0012_D.JPG
D:\G07_청도그레이스\CU_근접촬영\02_물리적손상잔디\20_고온피해\G07_CU_0220_DJI_20230823162143_0013_D.JPG
D:\G07_청도그레이스\CU_근접촬영\02_물리적손상잔디\20_고온피해\G07_CU_0220_DJI_20230823162145_0014_D.JPG
D:\G07_청도그레이스\CU_근접촬영\02_물리적손상잔디\20_고온피해\G07_CU_0220_DJI_20230823162147_0015_D.JPG
D:\G07_청도그레이스\CU_근접촬영\02_물리적손상잔디\20_고온피해\G07_CU_0220_DJI_20230823162149_0016_D.JPG
D:\G

In [16]:
InfoJsonArray

[{'id': '20230823162127_1286447220000_356634515556_160034000',
  '11originalFileJPG': 'G07_청도그레이스\\CU_근접촬영\\02_물리적손상잔디\\20_고온피해\\G07_CU_0220_DJI_20230823162127_0005_D.JPG',
  '11originalFileGrp': ['G07_청도그레이스\\CU_근접촬영\\02_물리적손상잔디\\20_고온피해\\G07_CU_0220_DJI_20230823162127_0005_MS_G.TIF',
   'G07_청도그레이스\\CU_근접촬영\\02_물리적손상잔디\\20_고온피해\\G07_CU_0220_DJI_20230823162127_0005_MS_NIR.TIF',
   'G07_청도그레이스\\CU_근접촬영\\02_물리적손상잔디\\20_고온피해\\G07_CU_0220_DJI_20230823162127_0005_MS_R.TIF',
   'G07_청도그레이스\\CU_근접촬영\\02_물리적손상잔디\\20_고온피해\\G07_CU_0220_DJI_20230823162127_0005_MS_RE.TIF'],
  'GeoTagInfo': {'coords': [128.644722, 35.663451555555554],
   'datetime_original': '2023:08:23 16:21:27',
   'gps_altitude': 160.034,
   'gps_altitude_ref': <GpsAltitudeRef.ABOVE_SEA_LEVEL: 0>},
  'label': {'level1': '02_물리적손상잔디',
   'level2': '20_고온피해',
   'level3': '',
   'TurfType': ''}},
 {'id': '20230823162129_1286447220278_356634490278_160052000',
  '11originalFileJPG': 'G07_청도그레이스\\CU_근접촬영\\02_물리적손상잔디\\20_고온피해\\G07_CU

Gather all data json files from Output Folder

In [18]:
out_folder = r'D:\ToAWS2'

In [19]:
files = glob.glob(os.path.join(out_folder,'**/*.json'), recursive=True)
files = [x for x in files if 'photo' in x]
files

['D:\\ToAWS2\\MGC001\\20230908\\photo.json',
 'D:\\ToAWS2\\MGC002\\20230823\\photo.json',
 'D:\\ToAWS2\\MGC002\\20230907\\photo.json',
 'D:\\ToAWS2\\MGC002\\20230919\\photo.json',
 'D:\\ToAWS2\\MGC002\\20230927\\photo.json',
 'D:\\ToAWS2\\MGC003\\20230909\\photo.json',
 'D:\\ToAWS2\\MGC004\\20230807\\photo.json',
 'D:\\ToAWS2\\MGC004\\20230822\\photo.json',
 'D:\\ToAWS2\\MGC004\\20230902\\photo.json',
 'D:\\ToAWS2\\MGC004\\20230905\\photo.json',
 'D:\\ToAWS2\\MGC004\\20230912\\photo.json',
 'D:\\ToAWS2\\MGC004\\20230919\\photo.json',
 'D:\\ToAWS2\\MGC005\\20230809\\photo.json',
 'D:\\ToAWS2\\MGC005\\20230817\\photo.json',
 'D:\\ToAWS2\\MGC005\\20230906\\photo.json',
 'D:\\ToAWS2\\MGC005\\20230927\\photo.json',
 'D:\\ToAWS2\\MGC006\\20230825\\photo.json',
 'D:\\ToAWS2\\MGC006\\20230911\\photo.json',
 'D:\\ToAWS2\\MGC007\\20230826\\photo.json',
 'D:\\ToAWS2\\MGC007\\20230907\\photo.json']

In [20]:
file_ = files[1]

In [21]:
json_dsDB = json.load(open(file_, encoding="utf8"))

In [22]:
[x for x in json_dsDB if x['id'] == InfoJsonArray[1]['id']]

[{'id': '20230823162129_1286447220278_356634490278_160052000',
  'originalFileJPG': 'original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\10_스타트퍼팅그린\\2023_0823_청도그레이스_스타트퍼팅그린_0.5m\\20230823_1621-1626_엽고성병,달라스팟,벤트,그린\\DJI_20230823162129_0006_D.JPG',
  'originalFileGrp': ['original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\10_스타트퍼팅그린\\2023_0823_청도그레이스_스타트퍼팅그린_0.5m\\20230823_1621-1626_엽고성병,달라스팟,벤트,그린\\DJI_20230823162129_0006_MS_G.TIF',
   'original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\10_스타트퍼팅그린\\2023_0823_청도그레이스_스타트퍼팅그린_0.5m\\20230823_1621-1626_엽고성병,달라스팟,벤트,그린\\DJI_20230823162129_0006_MS_NIR.TIF',
   'original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\10_스타트퍼팅그린\\2023_0823_청도그레이스_스타트퍼팅그린_0.5m\\20230823_1621-1626_엽고성병,달라스팟,벤트,그린\\DJI_20230823162129_0006_MS_R.TIF',
   'original\\2023_0823_청도그레이스_맑고뭉게구름_데이터 1차 분류본\\10_스타트퍼팅그린\\2023_0823_청도그레이스_스타트퍼팅그린_0.5m\\20230823_1621-1626_엽고성병,달라스팟,벤트,그린\\DJI_20230823162129_0006_MS_RE.TIF'],
  'destFolder': '20230823/10_스타트퍼팅그린/엽고성병,달라스팟,벤트,그린',
  'GeoTagInfo': {'coords

In [71]:
InfoJsonArray[100]

{'id': '20230823162145_1286447236944_356634455000_160026000',
 '11originalFileJPG': 'G07_청도그레이스\\CU_근접촬영\\05_병해잔디\\86_(조류)남조류\\G07_CU_0586_DJI_20230823162145_0014_D.JPG',
 '11originalFileGrp': ['G07_청도그레이스\\CU_근접촬영\\05_병해잔디\\86_(조류)남조류\\G07_CU_0586_DJI_20230823162145_0014_MS_G.TIF',
  'G07_청도그레이스\\CU_근접촬영\\05_병해잔디\\86_(조류)남조류\\G07_CU_0586_DJI_20230823162145_0014_MS_NIR.TIF',
  'G07_청도그레이스\\CU_근접촬영\\05_병해잔디\\86_(조류)남조류\\G07_CU_0586_DJI_20230823162145_0014_MS_R.TIF',
  'G07_청도그레이스\\CU_근접촬영\\05_병해잔디\\86_(조류)남조류\\G07_CU_0586_DJI_20230823162145_0014_MS_RE.TIF'],
 'GeoTagInfo': {'coords': [128.64472369444442, 35.6634455],
  'datetime_original': '2023:08:23 16:21:45',
  'gps_altitude': 160.026,
  'gps_altitude_ref': <GpsAltitudeRef.ABOVE_SEA_LEVEL: 0>},
 'label': {'level1': '05_병해잔디',
  'level2': '86_(조류)남조류',
  'level3': '',
  'TurfType': ''}}

In [ ]:
20230823162127_1286447972500_356633918611_161213000